In [1]:
from keras.applications import vgg16
model = vgg16.VGG16(weights=None, input_shape=[201, 201, 2], classes=17) # include_top=False to add features
# model.add(keras.layers...)

Using TensorFlow backend.
/usr/lib64/python3.6/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  str(input_shape[-1]) + ' input channels.')


In [2]:
from keras import backend as K

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        K.print_tensor(y_true)
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [3]:
model.compile(loss='categorical_crossentropy', metrics=[f1], optimizer=keras.optimizers.Adam(lr=0.1))

In [4]:
from dataGenerator import DataGenerator, TestDataGenerator
from personal_data import location_data
dataGenerator_train = DataGenerator(location_data, batch_size=128)
dataGenerator_val = DataGenerator(location_data, batch_size=128, isTrainingData=False)
dataGenerator_test = TestDataGenerator('../validation', batch_size=128)

In [5]:
from keras import callbacks
import time

# tensorboard --logdir ~/cytodata2019/tensorboard &
tensorboard = callbacks.TensorBoard(update_freq='batch',
    log_dir='../tensorboard/' + time.strftime("%Y%m%d-%H%M%S"))

In [6]:
model.fit_generator(dataGenerator_train.generate(),
    validation_data=dataGenerator_val.generate(),
    steps_per_epoch=len(dataGenerator_train),
    validation_steps=len(dataGenerator_val),
    callbacks=[tensorboard], verbose=1, epochs=1)

Epoch 1/1
78/78 [==============================] - 56s 713ms/step - loss: 2.8338 - f1: 0.0000e+00 - val_loss: 2.8288 - val_f1: 0.0000e+00


In [7]:
predictions = model.predict_generator(dataGenerator_test.generate(), steps=len(dataGenerator_test))

IndexError: index 128 is out of bounds for axis 0 with size 128

In [0]:
import numpy as np
hit_indices = np.argmax(predictions, axis=1).astype(dtype=np.int)
hit_names = [dataGenerator_train.labelsNames[i] for i in hit_indices]

In [0]:
cell_names = [s[26:-5] for s in dataGenerator_test.allData] 

In [0]:
print(len(cell_names), len(hit_names))
with open('output.csv', 'w') as file:
    file.write("cell_code,prediction\n")
    for (c,h) in zip(cell_names, hit_names):
        file.write(f"{c},{h}\n")
